ExtraTreesClassifier is the best classifier for this model, RandomForestClassifier is #2
Sometimes the Voting Classifier has a worse score, need to be careful to apply this. 
Setting the train data to a balanced between 0/1 really hurts several models => too many FP values
Trying different values for the RUS sampling_strategy, so far 0.1 seems okay. .002 is close to normal ratio in dataset. The higher the value, the faster the algorithm. Using .002 takes several hours, but using 1.0 takes a few minutes. KNN and other models will be heavily impacted by the amount of data modelled...

In [1]:
from mlens.preprocessing import Subset
import numpy as np
import datetime
import time
import pandas as pd

import sklearn.datasets
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

from sklearn.neighbors import  KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import ExtraTreesClassifier

from xgboost.sklearn import XGBClassifier
from lightgbm import LGBMClassifier
from collections import Counter
from imblearn.under_sampling import RandomUnderSampler
StartTime = datetime.datetime.now()


[MLENS] backend: threading


In [2]:
df = pd.read_csv('https://storage.googleapis.com/download.tensorflow.org/data/creditcard.csv')
X = df.loc[:, df.columns != 'Class']
y = df.loc[:, df.columns == 'Class']
y = np.array(y).ravel()

#dataset = sklearn.datasets.load_breast_cancer(return_X_y=False)
#X = pd.DataFrame(dataset.data, columns=dataset.feature_names)
#y = dataset.target 

x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.20,random_state=201, stratify=y)


In [3]:
sampling = 'true'

if (sampling == 'true'):
    # change sample based on this library: https://github.com/scikit-learn-contrib/imbalanced-learn
    # https://imbalanced-learn.org/stable/
    rus = RandomUnderSampler(random_state=0, sampling_strategy=0.01, replacement=False) # ss of 1.0 gives equally sized 0 and 1 counts
    x_train, y_train = rus.fit_resample(x_train, y_train)
    print(sorted(Counter(y_train).items()))
    x_train    
else:
    print("no sampling")


[(0, 39400), (1, 394)]


validate the splits has equal pct of 1 value

In [4]:
class MyTimer():
    # usage:
    #with MyTimer():                            
    #    rf.fit(X_train, y_train)
    
    def __init__(self):
        self.start = time.time()
    def __enter__(self):
        return self
    def __exit__(self, exc_type, exc_val, exc_tb):
        end = time.time()
        runtime = end - self.start
        msg = 'The function took {time} seconds to complete'
        print(msg.format(time=runtime))

In [5]:
dt_clf = DecisionTreeClassifier()
ab_clf = AdaBoostClassifier()
bc_clf = BaggingClassifier(n_estimators=10)
et_clf = ExtraTreesClassifier(n_estimators=10)
gbc_clf = GradientBoostingClassifier()
lgb_clf = LGBMClassifier()
gnb_clf = GaussianNB()
knn_clf = KNeighborsClassifier(n_neighbors=2)
#dbs_clf = DBSCAN(eps=3, min_samples=10)
svm_clf = SVC(gamma='scale', probability=True)
lr_clf  = LogisticRegression(solver='liblinear')
gb_clf  = GradientBoostingClassifier()
#gb_clf  = GradientBoostingClassifier(n_estimators=20, learning_rate=0.1, max_features=5, 
#                                    max_depth=3, random_state=None, subsample = 0.5, criterion='mse', 
#                                    min_samples_split = 10, min_samples_leaf = 10)

rf_clf  = RandomForestClassifier(n_estimators = 1000)
xgb_clf = XGBClassifier() # learning_rate =0.1, n_estimators=1000, max_depth=5, min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8, objective= 'binary:logistic', nthread=4, scale_pos_weight=1)


setup an ensemble using different classifiers, try to find a case where 2 classifiers beat 1

In [6]:
# add models here to the ensemble
voting_clf = VotingClassifier(
    estimators=[('knn', knn_clf), ('svc', svm_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (knn_clf, svm_clf, voting_clf):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 24.22335958480835 seconds to complete
KNeighborsClassifier 0.9982795547909132
0.5662122227900727
[[56851    13]
 [   85    13]]
SVC 0.9982795547909132
0.5
[[56864     0]
 [   98     0]]
VotingClassifier 0.9982795547909132
0.5662122227900727
[[56851    13]
 [   85    13]]
The function took 57.432759284973145 seconds to complete


Score is bad for all, and ensemble is no better

In [7]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('knn', knn_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (knn_clf, gb_clf, voting_clf):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 48.56696057319641 seconds to complete
KNeighborsClassifier 0.9982795547909132
0.5662122227900727
[[56851    13]
 [   85    13]]
GradientBoostingClassifier 0.9977002212000983
0.9020764186372354
[[56752   112]
 [   19    79]]
VotingClassifier 0.9984199992977775
0.622308293041471
[[56848    16]
 [   74    24]]
The function took 111.18328142166138 seconds to complete


Score is bad for all, and ensemble is worse

In [8]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('svc', svm_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (svm_clf, gb_clf, voting_clf):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 67.38122987747192 seconds to complete
SVC 0.9982795547909132
0.5
[[56864     0]
 [   98     0]]
GradientBoostingClassifier 0.9977879990168884
0.9021203831842247
[[56757   107]
 [   19    79]]
VotingClassifier 0.9982444436641972
0.8768827592939258
[[56788    76]
 [   24    74]]
The function took 143.0255491733551 seconds to complete


Score is bad for all, and ensemble is no better
svc is horrible here, can't predict the minority case at all

In [9]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('gb', gb_clf)], voting='soft')

wt1 = 4.0
wt0 = 1.0

with MyTimer(): 
    #voting_clf.fit(x_train, y_train, sample_weight=np.where(y_train == 1, wt1, wt0).flatten())
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (rf_clf, gb_clf, voting_clf):
        #if (clf.__class__.__name__ == "RandomForestClassifier"):
        #    print("found rf")
        #    clf.fit(x_train, y_train, sample_weight=np.where(y_train == 1, wt1, wt0).flatten())
        #else:
        #    print("not rf")
        #    clf.fit(x_train, y_train)
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 368.2173798084259 seconds to complete
RandomForestClassifier 0.999420666409185
0.9182178674790119
[[56847    17]
 [   16    82]]
GradientBoostingClassifier 0.9977704434535304
0.9021115902748269
[[56756   108]
 [   19    79]]
VotingClassifier 0.9980337769039008
0.9022434839157949
[[56771    93]
 [   19    79]]
The function took 765.7208182811737 seconds to complete


Score is good for RF only, and ensemble is much worse

In [10]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('gbc', gbc_clf), ('xgb', xgb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (gbc_clf, xgb_clf, voting_clf):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 61.94969034194946 seconds to complete
GradientBoostingClassifier 0.9976651100733822
0.9020588328184397
[[56750   114]
 [   19    79]]
XGBClassifier 0.9992802219023208
0.902867780483043
[[56842    22]
 [   19    79]]
VotingClassifier 0.9984375548611355
0.9024457208319457
[[56794    70]
 [   19    79]]
The function took 151.5291690826416 seconds to complete


Score is okay for xgb, and ensemble is no better

In [11]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('lgb', lgb_clf), ('gnb', gnb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (lgb_clf, gnb_clf, voting_clf):
        clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 2.382884979248047 seconds to complete
LGBMClassifier 0.9993504441557529
0.892716456306777
[[56848    16]
 [   21    77]]
GaussianNB 0.9932762192338752
0.8183686389581156
[[56516   348]
 [   35    63]]
VotingClassifier 0.9959973315543695
0.8706640189840709
[[56661   203]
 [   25    73]]
The function took 5.700852870941162 seconds to complete


Score is bad for all, and ensemble is a little better -- gnb is the only model to predict so many 1's

In [12]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('rf', rf_clf)], voting='soft')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (xgb_clf, rf_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 334.1916048526764 seconds to complete
XGBClassifier 0.9992802219023208
0.902867780483043
[[56842    22]
 [   19    79]]
RandomForestClassifier 0.999420666409185
0.913124619572083
[[56848    16]
 [   17    81]]
VotingClassifier 0.9993679997191109
0.9130982408438896
[[56845    19]
 [   17    81]]
The function took 668.3260383605957 seconds to complete


voila, score is good for both individually and slightly better for the ensemble (1 fewer wrong predictions)

In [13]:
with MyTimer(): 
    sw = np.where(y_train == 1, 4, 1).flatten()

# add models here
voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('rf', rf_clf)], voting='soft')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (xgb_clf, rf_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train, sample_weight=sw)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 0.0 seconds to complete
The function took 368.059366941452 seconds to complete
XGBClassifier 0.9991397773954567
0.9180771809286461
[[56831    33]
 [   16    82]]
RandomForestClassifier 0.9995435553526912
0.9182794178447969
[[56854    10]
 [   16    82]]
VotingClassifier 0.9993504441557529
0.9130894479344918
[[56844    20]
 [   17    81]]
The function took 637.7981021404266 seconds to complete


weights have an effect on XGB, causing more FP than without 

In [14]:
with MyTimer(): 
    sw = np.where(y_train == 1, 1, 4).flatten()

# add models here
voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('rf', rf_clf)], voting='soft')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (xgb_clf, rf_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train, sample_weight=sw)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 0.0 seconds to complete
The function took 323.5075113773346 seconds to complete
XGBClassifier 0.9993153330290369
0.8876056225810527
[[56847    17]
 [   22    76]]
RandomForestClassifier 0.999420666409185
0.913124619572083
[[56848    16]
 [   17    81]]
VotingClassifier 0.9993153330290369
0.8926988704879814
[[56846    18]
 [   21    77]]
The function took 686.2084558010101 seconds to complete


In [15]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('rf', rf_clf)], voting='hard')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (xgb_clf, rf_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 355.15858721733093 seconds to complete
XGBClassifier 0.9992802219023208
0.902867780483043
[[56842    22]
 [   19    79]]
RandomForestClassifier 0.999420666409185
0.9182178674790119
[[56847    17]
 [   16    82]]
VotingClassifier 0.999420666409185
0.9029381237582259
[[56850    14]
 [   19    79]]
The function took 699.4767827987671 seconds to complete


soft voting gives better results here

try 2 more sets of models

In [16]:

# add models here
voting_clf = VotingClassifier(
    estimators=[('dt', dt_clf), ('ab', ab_clf)], voting='soft')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (dt_clf, ab_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 13.398539304733276 seconds to complete
DecisionTreeClassifier 0.9977704434535304
0.9173913339956129
[[56753   111]
 [   16    82]]
AdaBoostClassifier 0.9992451107756047
0.907943442571176
[[56839    25]
 [   18    80]]
VotingClassifier 0.9977353323268143
0.9173737481768172
[[56751   113]
 [   16    82]]
The function took 25.79205322265625 seconds to complete


In [17]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('bc', bc_clf), ('et',et_clf)], voting='soft')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (bc_clf, et_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 19.32780933380127 seconds to complete
BaggingClassifier 0.9993504441557529
0.9181826958414203
[[56843    21]
 [   16    82]]
ExtraTreesClassifier 0.9995611109160493
0.9081017149403374
[[56857     7]
 [   18    80]]
VotingClassifier 0.9994557775359011
0.8978624616700929
[[56853    11]
 [   20    78]]
The function took 37.19050312042236 seconds to complete


bc and et have good results, combine them with xgb and rf so we have 4 models to use


In [18]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('xgb', xgb_clf), ('bc', bc_clf), ('et',et_clf)], voting='soft')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (xgb_clf, bc_clf, et_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 27.85802388191223 seconds to complete
XGBClassifier 0.9992802219023208
0.902867780483043
[[56842    22]
 [   19    79]]
BaggingClassifier 0.9992099996488887
0.9130191046593088
[[56836    28]
 [   17    81]]
ExtraTreesClassifier 0.9994557775359011
0.9029557095770216
[[56852    12]
 [   19    79]]
VotingClassifier 0.9993679997191109
0.9130982408438896
[[56845    19]
 [   17    81]]
The function took 59.76839470863342 seconds to complete


In [19]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('xgb', xgb_clf), ('bc', bc_clf), ('et', et_clf)], voting='soft')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (rf_clf, xgb_clf, bc_clf, et_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 361.6446259021759 seconds to complete
RandomForestClassifier 0.999403110845827
0.9131158266626853
[[56847    17]
 [   17    81]]
XGBClassifier 0.9992802219023208
0.902867780483043
[[56842    22]
 [   19    79]]
BaggingClassifier 0.999385555282469
0.908013785846359
[[56847    17]
 [   18    80]]
ExtraTreesClassifier 0.9994382219725431
0.8927604208537663
[[56853    11]
 [   21    77]]
VotingClassifier 0.999385555282469
0.9131070337532875
[[56846    18]
 [   17    81]]
The function took 690.471239566803 seconds to complete


In [20]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('xgb', xgb_clf), ('bc', bc_clf), ('et', et_clf)], voting='hard')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (rf_clf, xgb_clf, bc_clf, et_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 330.05251955986023 seconds to complete
RandomForestClassifier 0.9994382219725431
0.9182266603884097
[[56848    16]
 [   16    82]]
XGBClassifier 0.9992802219023208
0.902867780483043
[[56842    22]
 [   19    79]]
BaggingClassifier 0.9992099996488887
0.8977393609385229
[[56839    25]
 [   20    78]]
ExtraTreesClassifier 0.9994382219725431
0.8978536687606951
[[56852    12]
 [   20    78]]
VotingClassifier 0.999385555282469
0.9029205379394302
[[56848    16]
 [   19    79]]
The function took 692.7915947437286 seconds to complete


good improvement, 3 fewer errors by using voting...

In [21]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('et', et_clf)], voting='soft')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (rf_clf, et_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 292.0507206916809 seconds to complete
RandomForestClassifier 0.9994382219725431
0.9131334124814809
[[56849    15]
 [   17    81]]
ExtraTreesClassifier 0.9994557775359011
0.8978624616700929
[[56853    11]
 [   20    78]]
VotingClassifier 0.9995084442259752
0.9131685841190725
[[56853    11]
 [   17    81]]
The function took 583.407958984375 seconds to complete


In [22]:
# add models here
voting_clf = VotingClassifier(
    estimators=[('rf', rf_clf), ('et', et_clf)], voting='hard')
    #estimators=[('lr', lr_clf), ('svc', svm_clf), ('rf', rf_clf), ('gb', gb_clf)], voting='soft')

with MyTimer(): 
    voting_clf.fit(x_train, y_train)

# compare single models to ensemble voting model
# add models here
with MyTimer(): 
    for clf in (rf_clf, et_clf, voting_clf):
        #clf.fit(x_train, y_train, sample_weight=sw, preprocessing=preprocess_cases)
        clf.fit(x_train, y_train)
        #clf.fit(x_train, y_train)
        y_pred = clf.predict(x_test)
        print(clf.__class__.__name__, accuracy_score(y_test, y_pred))
        print(roc_auc_score(y_test, y_pred))
        print(confusion_matrix(y_test, y_pred))

The function took 282.4328303337097 seconds to complete
RandomForestClassifier 0.999403110845827
0.9131158266626853
[[56847    17]
 [   17    81]]
ExtraTreesClassifier 0.9994382219725431
0.8978536687606951
[[56852    12]
 [   20    78]]
VotingClassifier 0.9994382219725431
0.8927604208537663
[[56853    11]
 [   21    77]]
The function took 699.7399299144745 seconds to complete


In [23]:
print("Start: ", StartTime)
print("End: ", datetime.datetime.now())

Start:  2020-07-06 12:40:17.382809
End:  2020-07-06 15:27:24.600744
